In [1]:
from langchain.chains import LLMChain, ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers import CommaSeparatedListOutputParser

# Conversational Chain (Memory)

In [3]:
output_parser = CommaSeparatedListOutputParser()
conversation = ConversationChain(
    llm=ChatGoogleGenerativeAI(model = 'gemini-pro', temperature=0),
    memory=ConversationBufferMemory()
)

conversation.predict(input="List all possible words as substitute for 'artificial' as comma separated.")

'synthetic, man-made, fabricated, simulated, fake, ersatz, pseudo, artificial'

In [4]:
conversation.predict(input="And the next 4?")

'unnatural, contrived, forced, affected'

# Sequential Chain

In [5]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[conversation, conversation])

# Custom Chain

In [8]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain
from typing import Dict, List
from langchain import PromptTemplate

In [9]:
llm=ChatGoogleGenerativeAI(model = 'gemini-pro', temperature=0)

In [7]:
class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)
    
    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']
    
    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}


In [11]:
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("education")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:
Education is the process of facilitating learning, or the acquisition of knowledge, skills, values, beliefs, and habits. It is the act or process of imparting or acquiring general knowledge, developing the powers of reasoning and judgment, and generally of preparing oneself or others intellectually for mature life.

Education can take place in a formal setting, such as a school or university, or in an informal setting, such as at home or in the workplace. It can be provided by teachers, parents, peers, or other individuals.

The purpose of education is to help people develop the skills and knowledge they need to succeed in life. This includes academic skills, such as reading, writing, and mathematics, as well as social and emotional skills, such as cooperation, communication, and problem-solving. Education can also help people develop critical thinking skills and the ability to learn new things throughout their lives.

There are many different types of education, e